In [ ]:
from IPython.display import clear_output
from kendama_env import KendamaEnv
import numpy as np 
import time
env = KendamaEnv()

a = 0
start_time = time.time()
while 1:
    current_time = time.time()
    elapsed_time = current_time - start_time

    if elapsed_time > 1.0/240:
        a += 0.005
        start_time = time.time()
        # action = [list(np.random.normal(scale = 0,loc=1,size = 3)),list(np.random.normal(scale = 10,size = 3))]
        action = [0,0,0.1*np.sin(20*a),0,0,0]
        
        clear_output(wait=True)
        
        ob, reward, done, _ = env.step(action)

        if(done):
            env.reset()

In [1]:
import gym
from IPython.display import clear_output
from kendama_env import KendamaEnv
import numpy as np 
import time
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import SubprocVecEnv
from stable_baselines import PPO2,SAC
from stable_baselines.common.vec_env import VecNormalize
from IPython.display import clear_output

def make_env( rank, seed=0):
    """
    Utility function for multiprocessed env.

    :param env_id: (str) the environment ID
    :param num_env: (int) the number of environments you wish to have in subprocesses
    :param seed: (int) the inital seed for RNG
    :param rank: (int) index of the subprocess
    """
    def _init():
        env = KendamaEnv(render=(rank==0))
        return env
    
    return _init

if __name__ == '__main__':
    num_cpu = 8
    env = SubprocVecEnv([make_env( i) for i in range(num_cpu)])
    #env = VecNormalize(env, norm_reward= False)
    
    
    env = VecNormalize.load("vec_normalize.pkl", env)
    env.norm_reward = False

    #model = PPO2(MlpPolicy, env, verbose=0,tensorboard_log="./log_model/", gamma=0.985)
    model = PPO2.load(".ppo2.zip",env, verbose=0,tensorboard_log="./log_model2/", gamma=0.985)


    #model.learn(total_timesteps=10000000)

W0308 13:33:03.095343 13728 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0308 13:33:17.036575 13728 module_wrapper.py:137] From C:\Users\arnau\Anaconda3\lib\site-packages\stable_baselines\common\tf_util.py:58: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0308 13:33:17.222079 13728 deprecation.py:323] From C:\Users\arnau\Anaconda3\lib\site-packages\stable_baselines\common\policies.py:560: flatten (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.flatten instead.
W0308 13:33:17.224074 13728 deprecation.py:323] From C:\Users\arnau

In [ ]:
if __name__ == '__main__':
    model.learn(total_timesteps=20000000,tb_log_name="lavictoiredesesgrandsmorts", reset_num_timesteps=False)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./log_model/


In [ ]:
obs = env.reset()
start_time = time.time()
while 1:
    current_time = time.time()
    elapsed_time = current_time - start_time
    if elapsed_time > 1.0/120:
        start_time=current_time
        action, _states = model.predict(obs)
        obs, rewards, dones, info = env.step(action)
        if dones[0]:
            obs = env.reset()


In [3]:
import os
log_dir = "."
model.save(log_dir + "ppo2")
stats_path = os.path.join(log_dir, "vec_normalize.pkl")
env.save(stats_path)